## Part 2: Replacing missing values and adding new features

(See Part1 for 'Exploration of training dataset'). 

Titanic challenge problem (https://www.kaggle.com/c/titanic/) offers an oppurtunity to practice your data mining and machine learning skills at a beginners level but with interesting challenges.
The main goal of this challenge is to predict the fate of each passenger given basic information on his/her status.

In this part, I will try to polish the data and replace the missing values in the input and test data. We also try to learn from 'Name' of passengers to extract the social status of each passenger. This will add new features to our modeling efforts which will discussed in part 3. 

We need: 

- Pandas (Statistical analysis)

- Numpy (numerical package) 

- Matplotlib (plotting)

In [29]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Before finding the missing values, we combine the train and test data. We do not need the 'Survived' column. 

In [30]:
dtrain = pd.read_csv('train.csv')
dtest = pd.read_csv('test.csv')
dtrain.drop('Survived',axis=1, inplace = True)
data = pd.concat([dtrain,dtest])
data.to_csv('alldata.csv',index=False)

In [31]:
data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [32]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    1309 non-null int64
Pclass         1309 non-null int64
Name           1309 non-null object
Sex            1309 non-null object
Age            1046 non-null float64
SibSp          1309 non-null int64
Parch          1309 non-null int64
Ticket         1309 non-null object
Fare           1308 non-null float64
Cabin          295 non-null object
Embarked       1307 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 122.7+ KB


We have:

- 263 missing data in the 'Age' column. 

- 1 missing data in the 'Fare' column. 

- 2 missing data in the 'Embarked' column. 

- 1014 missing data in the 'Cabin' column. 


## 2.1 Replacing missing 'Age' data ##

We can take a closer look at the 'Name' column. This feature seems to include useful information that can be employed to assign the missing values for the 'Age' as well as the social status of the passengers. Let's take a closer look by extracting the titles for each passenger. We add a new column called 'Title' for each passenger:  

In [33]:
# Extract titles in Name column
data['Title'] = data['Name'].map(lambda name:name.split(',')[1].split('.')[0].strip())
data['Title'].unique()

array(['Mr', 'Mrs', 'Miss', 'Master', 'Don', 'Rev', 'Dr', 'Mme', 'Ms',
       'Major', 'Lady', 'Sir', 'Mlle', 'Col', 'Capt', 'the Countess',
       'Jonkheer', 'Dona'], dtype=object)

Now we can map these extracted titles to more general categories: 

In [34]:
# Map the extracted titles to different categories
def assign_Titles(df_in):
    df = df_in
    Title_Category = {
                    "Jonkheer":   "Royalty",
                    "Don":        "Royalty",
                    "Sir" :       "Royalty",
                    "Lady" :      "Royalty",
                    "the Countess":"Royalty",
                    "Dona":       "Royalty",
                    "Capt":       "HighRanked",
                    "Col":        "HighRanked",
                    "Major":      "HighRanked",
                    "Dr":         "Dr",
                    "Rev":        "Rev",
                    "Mme":        "Mrs",
                    "Mrs" :       "Mrs",
                    "Mlle":       "Miss",
                    "Ms":         "Miss",
                    "Miss" :      "Miss",
                    "Mr" :        "Mr",
                    "Master" :    "Master"
    }
    df['Title'] = df.Title.map(Title_Category)
    return df

In [11]:
# Now group different titles into categories based on social class and/or occupation
data = assign_Titles(data)
data['Title'].unique()

array(['Mr', 'Mrs', 'Miss', 'Master', 'Royalty', 'Rev', 'Dr', 'HighRanked'], dtype=object)

It would be useful to check the ticket class and the social class of passengers:  

In [35]:
grouped = data.groupby(['Pclass','Title'])
grouped.median()

PassengerId   Age  SibSp  Parch       Fare
Pclass Title                                                   
1      Capt                746.0  70.0    1.0    1.0   71.00000
       Col                 859.0  54.5    0.0    0.0   32.00000
       Don                  31.0  40.0    0.0    0.0   27.72080
       Dona               1306.0  39.0    0.0    0.0  108.90000
       Dr                  714.0  49.0    0.5    0.0   60.72915
       Jonkheer            823.0  38.0    0.0    0.0    0.00000
       Lady                557.0  48.0    1.0    0.0   39.60000
       Major               493.5  48.5    0.0    0.0   28.52500
       Master              803.0   6.0    1.0    2.0  134.50000
       Miss                513.0  30.0    0.0    0.0  108.65415
       Mlle                676.5  24.0    0.0    0.0   59.40210
       Mme                 370.0  24.0    0.0    0.0   69.30000
       Mr                  634.0  41.5    0.0    0.0   47.10000
       Mrs                 857.0  45.0    1.0    0.0   78.26670
       Sir                 600.0  49.0    1.0    0.0   56.92920
       the Countess        760.0  33.0    0.0    0.0   86.50000
2      Dr                  358.5  38.5    0.0    0.0   12.25000
       Master              550.0   2.0    1.0    1.0   26.00000
       Miss                606.5  20.0    0.0    0.0   20.25000
       Mr                  723.5  30.0    0.0    0.0   13.00000
       Mrs                 547.0  30.5    1.0    0.0   26.00000
       Ms                  444.0  28.0    0.0    0.0   13.00000
       Rev                 738.0  41.5    0.0    0.0   13.00000
3      Master              789.0   6.0    3.0    1.0   22.35830
       Miss                603.5  18.0    0.0    0.0    8.05000
       Mr                  640.5  26.0    0.0    0.0    7.89580
       Mrs                 658.0  31.0    1.0    1.0   15.50000
       Ms                  980.0   NaN    0.0    0.0    7.75000

#### Royalty and HighRanked: Only on Pclass = 1
#### Rev: Only on Pclass = 2

#### We now replace the missing age-value by the median for each 'Title' on each 'Pclass'.

In [36]:
def assign_missing_Age(df_in):
    df = df_in
    for idx in range(0,len(df)):
        pass_age = df['Age'].values[idx]
        if np.isnan(pass_age):
            val=0
            if (df['Pclass'].values[idx] == 1):
                if (df['Title'].values[idx] == "Dr"):
                    val = 49
                elif (df['Title'].values[idx] == "HighRanked"):
                    val = 53
                elif (df['Title'].values[idx] == "Master"):
                    val = 6
                elif (df['Title'].values[idx] == "Miss"):
                    val = 30
                elif (df['Title'].values[idx] == "Mr"):
                    val = 41.5
                elif (df['Title'].values[idx] == "Mrs"):
                    val = 45
                elif (df['Title'].values[idx] == "Royalty"):
                    val = 39.5
            elif (df['Pclass'].values[idx] == 2):
                if (df['Title'].values[idx] == "Dr"):
                    val = 38.5
                elif (df['Title'].values[idx] == "Master"):
                    val = 2
                elif (df['Title'].values[idx] == "Miss"):
                    val = 20
                elif (df['Title'].values[idx] == "Mr"):
                    val = 30
                elif (df['Title'].values[idx] == "Mrs"):
                    val = 30.5
                elif (df['Title'].values[idx] == "Rev"):
                    val = 41.5
            elif (df['Pclass'].values[idx] == 3):
                if (df['Title'].values[idx] == "Master"):
                    val = 6
                elif (df['Title'].values[idx] == "Miss"):
                    val = 18
                elif (df['Title'].values[idx] == "Mr"):
                    val = 26
                elif (df['Title'].values[idx] == "Mrs"):
                    val = 31
            df['Age'].values[idx] = val
    return df

In [37]:
data = assign_missing_Age(data)

In [38]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 12 columns):
PassengerId    1309 non-null int64
Pclass         1309 non-null int64
Name           1309 non-null object
Sex            1309 non-null object
Age            1309 non-null float64
SibSp          1309 non-null int64
Parch          1309 non-null int64
Ticket         1309 non-null object
Fare           1308 non-null float64
Cabin          295 non-null object
Embarked       1307 non-null object
Title          1309 non-null object
dtypes: float64(2), int64(4), object(6)
memory usage: 132.9+ KB


## 2.2 Replacing missing data: 'Fare'
There is only one missing data in the 'Fare' column. We assign the median ticket fare based on the passenger's Pclass. 

In [39]:
def assign_missing_Fare(df_in):
    df = df_in
    df['Fare'].fillna(df.groupby('Pclass')['Fare'].transform('median'), inplace=True)
    return df

In [40]:
data = assign_missing_Fare(data)

In [41]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 12 columns):
PassengerId    1309 non-null int64
Pclass         1309 non-null int64
Name           1309 non-null object
Sex            1309 non-null object
Age            1309 non-null float64
SibSp          1309 non-null int64
Parch          1309 non-null int64
Ticket         1309 non-null object
Fare           1309 non-null float64
Cabin          295 non-null object
Embarked       1307 non-null object
Title          1309 non-null object
dtypes: float64(2), int64(4), object(6)
memory usage: 132.9+ KB


In [42]:
data = assign_missing_Fare(data)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 12 columns):
PassengerId    1309 non-null int64
Pclass         1309 non-null int64
Name           1309 non-null object
Sex            1309 non-null object
Age            1309 non-null float64
SibSp          1309 non-null int64
Parch          1309 non-null int64
Ticket         1309 non-null object
Fare           1309 non-null float64
Cabin          295 non-null object
Embarked       1307 non-null object
Title          1309 non-null object
dtypes: float64(2), int64(4), object(6)
memory usage: 132.9+ KB


## 2.3 Assign missing data: 'Embarked'
There are two missing data in the 'Embarked' column. 
There seems to be a correlation between the ticket fare and the point of embarkment. 

In [43]:
data[data['Embarked'].isnull()]

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
61,62,1,"Icard, Miss. Amelie",female,38.0,0,0,113572,80.0,B28,NaN,Miss
829,830,1,"Stone, Mrs. George Nelson (Martha Evelyn)",female,62.0,0,0,113572,80.0,B28,NaN,Mrs


In [44]:
group = data.groupby(['Embarked'])
group.mean()

,PassengerId,Pclass,Age,SibSp,Parch,Fare
Embarked,,,,,,
C,690.655556,1.851852,31.601556,0.400000,0.370370,62.336267
Q,668.593496,2.894309,24.735772,0.341463,0.113821,12.409012
S,643.095186,2.347921,29.121171,0.550328,0.426696,27.397633


Both entries are purchased for $80 which places them in the "C" embarked location. 

In [45]:
def assign_missing_Embarked(df_in):
    df = df_in
    # Two missing entries in the 'Embarked' column.
    index = df['Embarked'].index[df['Embarked'].isnull()]
    # seems like there is a correlation between Ticket fare and Embarked
    # C: Most expensive (>60), S: Average Q: Cheapest ticket (<12)
    df['Embarked'].values[index] = "C"
    return df

In [46]:
data = assign_missing_Embarked(data)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 12 columns):
PassengerId    1309 non-null int64
Pclass         1309 non-null int64
Name           1309 non-null object
Sex            1309 non-null object
Age            1309 non-null float64
SibSp          1309 non-null int64
Parch          1309 non-null int64
Ticket         1309 non-null object
Fare           1309 non-null float64
Cabin          295 non-null object
Embarked       1309 non-null object
Title          1309 non-null object
dtypes: float64(2), int64(4), object(6)
memory usage: 132.9+ KB


## 2.4 Finalizing the data

We are almost ready to use the finalized dataset. For now, let us not use the 'Cabin' column as a feature. 
Although, one can always do more research to extract useful information. 

We now convert the categorical data to numbered logical values. There are benefits to this when we perform any regression or classification.

In [47]:
# Convert categorical features to sub-features with (0 or 1) values. 
# Example: 
# Applying this function on dataset with ('Sex': male or female)  will produce two new columns
# Sex_male (0 or 1)
# Sex_female (0 or 1)
def category_to_dummies(df, which_column):
    
    class_dummies = pd.get_dummies(df[which_column],prefix=which_column)
    # adding dummy variables
    combined = pd.concat([df,class_dummies],axis=1)
    # removing the original feature
    combined.drop(which_column,axis=1,inplace=True)
    return combined

In [48]:
data = category_to_dummies(data, 'Pclass')
data = category_to_dummies(data, 'Title')
data = category_to_dummies(data, 'Embarked')
data = category_to_dummies(data, 'Sex')

In [49]:
# Drop these columns. We won't need them anymore. 
data.drop('Ticket', axis=1, inplace=True)
data.drop('Name', axis=1, inplace=True)
data.drop('Cabin', axis=1, inplace=True)

In [50]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 31 columns):
PassengerId           1309 non-null int64
Age                   1309 non-null float64
SibSp                 1309 non-null int64
Parch                 1309 non-null int64
Fare                  1309 non-null float64
Pclass_1              1309 non-null float64
Pclass_2              1309 non-null float64
Pclass_3              1309 non-null float64
Title_Capt            1309 non-null float64
Title_Col             1309 non-null float64
Title_Don             1309 non-null float64
Title_Dona            1309 non-null float64
Title_Dr              1309 non-null float64
Title_Jonkheer        1309 non-null float64
Title_Lady            1309 non-null float64
Title_Major           1309 non-null float64
Title_Master          1309 non-null float64
Title_Miss            1309 non-null float64
Title_Mlle            1309 non-null float64
Title_Mme             1309 non-null float64
Title_Mr            

In [51]:
data.to_csv("polished.csv", index=False)